#Topic modeling for articles title

In [41]:
import pandas as pd

data = pd.read_csv('train.csv',engine='python')
data = data.TITLE.to_list()
data[:5]

['Reconstructing Subject-Specific Effect Maps',
 'Rotation Invariance Neural Network',
 'Spherical polyharmonics and Poisson kernels for polyharmonic functions',
 'A finite element approximation for the stochastic Maxwell--Landau--Lifshitz--Gilbert system',
 'Comparative study of Discrete Wavelet Transforms and Wavelet Tensor Train decomposition to feature extraction of FTIR data of medicinal plants']

Предобработаем текст

In [51]:
import nltk
import string
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
nltk.download('punkt')
porter = PorterStemmer()
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]


def text_preprocess(text):
  text = text.lower()
  text_p = "".join([char for char in text if char not in string.punctuation])
  words = word_tokenize(text_p)
  filtered_words = [word for word in words if word not in stop_words]
  stemmed = [porter.stem(word) for word in filtered_words]
  
  return stemmed

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [52]:
data = [text_preprocess(text) for text in data]

In [55]:
data[:3]

[['reconstruct', 'subjectspecif', 'effect', 'map'],
 ['rotat', 'invari', 'neural', 'network'],
 ['spheric', 'polyharmon', 'poisson', 'kernel', 'polyharmon', 'function']]

## Bert

In [16]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Тексты для берта надо склеить обратно.


In [56]:
def connect_data_for_bert(tokens):
  text = ''
  for token in tokens:
    text+=token+' '
  return text
data_bert = [connect_data_for_bert(text) for text in data]

In [58]:
data_bert[:3]

['reconstruct subjectspecif effect map ',
 'rotat invari neural network ',
 'spheric polyharmon poisson kernel polyharmon function ']

In [61]:
from bertopic import BERTopic
model = BERTopic(verbose=True,embedding_model='paraphrase-MiniLM-L3-v2', nr_topics =7)
headline_topics, _ = model.fit_transform(data_bert)

Batches:   0%|          | 0/656 [00:00<?, ?it/s]

2023-03-11 16:07:55,409 - BERTopic - Transformed documents to Embeddings
2023-03-11 16:08:10,565 - BERTopic - Reduced dimensionality
2023-03-11 16:08:13,178 - BERTopic - Clustered reduced embeddings
2023-03-11 16:08:23,986 - BERTopic - Reduced number of topics from 240 to 7


In [62]:
model.get_topic_info()

,Topic,Count,Name
0,-1,9344,-1_model_learn_use_network
1,0,10960,0_network_learn_model_gener
2,1,311,1_tree_statist_forest_lowrank
3,2,156,2_game_gan_gametheoret_equilibria
4,3,149,3_causal_brain_neuron_eeg
5,4,31,4_datadriven_framework_estim_optim
6,5,21,5_industri_analysi_40_largescal


In [66]:
model.get_document_info(data_bert)

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,reconstruct subjectspecif effect map,-1,-1_model_learn_use_network,model - learn - use - network - system - estim...,0.000000,False
1,rotat invari neural network,0,0_network_learn_model_gener,network - learn - model - gener - neural - use...,0.775822,False
2,spheric polyharmon poisson kernel polyharmon f...,-1,-1_model_learn_use_network,model - learn - use - network - system - estim...,0.000000,False
3,finit element approxim stochast maxwelllandaul...,-1,-1_model_learn_use_network,model - learn - use - network - system - estim...,0.000000,False
4,compar studi discret wavelet transform wavelet...,0,0_network_learn_model_gener,network - learn - model - gener - neural - use...,0.980778,False
...,...,...,...,...,...,...
20967,contemporari machin learn guid practition phys...,0,0_network_learn_model_gener,network - learn - model - gener - neural - use...,0.919745,False
20968,uniform diamond coat wcco hard alloy cut inser...,-1,-1_model_learn_use_network,model - learn - use - network - system - estim...,0.000000,False
20969,analys soccer game cluster conceptor,0,0_network_learn_model_gener,network - learn - model - gener - neural - use...,0.170206,False
20970,effici simul lefttail sum correl lognorm variat,-1,-1_model_learn_use_network,model - learn - use - network - system - estim...,0.000000,False


#LDA

In [71]:
import gensim

tm_dictionary2 = gensim.corpora.Dictionary(data)
#tm_dictionary2.filter_extremes(no_above=0.1, no_below=4)
#tm_dictionary2.compactify()

corpus = [tm_dictionary2.doc2bow(text) for text in data]
lda = gensim.models.LdaMulticore(corpus, 
                                 num_topics = 6, # число топиков
                                 id2word=tm_dictionary2, 
                                 passes=10)


In [73]:
lda.print_topics()

[(0,
  '0.013*"imag" + 0.012*"data" + 0.011*"use" + 0.011*"learn" + 0.009*"social" + 0.009*"detect" + 0.008*"predict" + 0.007*"toward" + 0.007*"analysi" + 0.006*"dark"'),
 (1,
  '0.050*"network" + 0.039*"learn" + 0.025*"model" + 0.020*"neural" + 0.019*"deep" + 0.017*"use" + 0.013*"gener" + 0.010*"graph" + 0.009*"data" + 0.008*"analysi"'),
 (2,
  '0.011*"studi" + 0.011*"system" + 0.008*"control" + 0.007*"robot" + 0.006*"model" + 0.005*"stabil" + 0.005*"synthesi" + 0.005*"machin" + 0.005*"separ" + 0.004*"translat"'),
 (3,
  '0.017*"estim" + 0.013*"optim" + 0.011*"space" + 0.010*"stochast" + 0.010*"model" + 0.009*"distribut" + 0.008*"function" + 0.008*"problem" + 0.008*"applic" + 0.007*"algorithm"'),
 (4,
  '0.011*"optim" + 0.010*"system" + 0.008*"comput" + 0.008*"use" + 0.008*"matrix" + 0.008*"model" + 0.008*"power" + 0.008*"process" + 0.007*"problem" + 0.007*"effici"'),
 (5,
  '0.013*"quantum" + 0.011*"field" + 0.010*"equat" + 0.009*"magnet" + 0.009*"group" + 0.008*"dynam" + 0.008*"alge

#NMF

In [78]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

vectorizer = TfidfVectorizer(max_features=1500, min_df=10, stop_words='english')
X = vectorizer.fit_transform(data_bert)
words = np.array(vectorizer.get_feature_names_out())

nmf = NMF(n_components=6, solver="mu")
W = nmf.fit_transform(X)
H = nmf.components_

for i, topic in enumerate(H):
     print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in words[topic.argsort()[-10:]]])))


Topic 1: languag,mixtur,linear,use,select,infer,data,bayesian,predict,model
Topic 2: predict,adversari,detect,train,use,recurr,convolut,deep,neural,network
Topic 3: onlin,featur,transfer,classif,use,represent,reinforc,machin,deep,learn
Topic 4: space,graph,structur,dynam,equat,function,use,data,estim,analysi
Topic 5: train,geometri,music,rel,test,net,function,theorem,adversari,gener
Topic 6: design,gradient,bayesian,problem,approach,distribut,stochast,algorithm,control,optim


#Top2Vec

In [80]:
!pip install top2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [84]:
from top2vec import Top2Vec

umap_args = {'n_neighbors': 10,
             'n_components': 6,
             'metric': 'cosine',
             "random_state": 42}

hdbscan_args = {'min_cluster_size': 10,
                'min_samples':5,
                'metric': 'euclidean',
                'cluster_selection_method': 'eom'}

model = Top2Vec(documents= data_bert, speed='deep-learn', workers=8, min_count = 0, embedding_model='distiluse-base-multilingual-cased', umap_args = umap_args, hdbscan_args = hdbscan_args)
model.get_num_topics()

2023-03-11 17:12:43,669 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
2023-03-11 17:12:45,173 - top2vec - INFO - Downloading distiluse-base-multilingual-cased model
INFO:top2vec:Downloading distiluse-base-multilingual-cased model
2023-03-11 17:12:50,028 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2023-03-11 17:48:05,947 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2023-03-11 17:48:23,548 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2023-03-11 17:48:28,088 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


291

In [94]:
model.hierarchical_topic_reduction(num_topics=6)
for i in range(6):
  print(model.topic_words_reduced[i])

['manifoldvalu' 'quasigeostroph' 'orbitaldepend' 'symplectomorph'
 'quasihomogen' 'quasiparticl' 'spectrumpreserv' 'ultraspheroid'
 'multistellar' 'multigranular' 'multidirichlet' 'orbitalupd'
 'multiphasefield' 'quasitriangular' 'quasiarithmet' 'discretemargin'
 'amplitudetophas' 'multigrasp' 'multisymplect' 'inftygroupoid' 'manifold'
 'varianceprevari' 'asynchronytoler' 'covarianceinsur' 'quasinonexpans'
 'submultitil' 'saturatedfrag' 'sigmapisigma' 'diffractionawar'
 'orbitalact' 'variancereduct' 'semigeostroph' 'spatiallyresolv'
 'spectrallynorm' 'pseudospher' 'gammadiverg' 'quasiintegr'
 'deepstrongcoupl' 'quasihereditari' 'heteronuclear' 'subtomogram'
 'heterointerfac' 'ultradiscret' 'fractaltyp' 'orbitalfre' 'sparselearn'
 'variancereduc' 'hyperbolicequ' 'multihybrid' 'epitwodimension']
['algorithminvari' 'symmetryenforc' 'symmetricnmf' 'linearithm'
 'linftyalgebra' 'pseudosymmetr' 'semialgebra' 'exponentialtyp'
 'asymmetryinduc' 'symmetrybreak' 'polylogarithm' 'pcalgorithm'
 'd